In [ ]:
# load init moudle file

load "/work/pannote/panbot_note.rb"
nil

In [ ]:
puts "==status count=="
puts Task.group(:status).count
puts "==running task=="



puts Task.where(status:"run").map {|x| [x.id,x.name,x.runner,Time.now-x.updated_at].join(" | ")}.join("\n")


==status count==
{"run"=>1, "close"=>60}
==running task==
3147 | loop-1 | panworker-main2-13 | 25.275864442


In [ ]:
def abort_reset_to_open()
  Task.where(status:"abort").map {|x| x.status="open"; x.save}
end

def task_clean()
  Task.destroy_all
end

def task_hello_task(count)
  count.times {|x|
    code = <<~TASKCODE

    puts "hello world!"
    _log "hello world!\n"
    
    TASKCODE
    Task.create_task("hello world task-#{x}", code)
  }
end


:task_hello_task

In [ ]:
abort_reset_to_open

[true, true, true, true, true, true, true, true]

In [ ]:
puts Task.where(status:"abort").order(:id).map {|x| [x.runner] }

panworker-main2-28
panworker-main2-13


In [ ]:
puts Task.where(status:"abort").order(:id).map {|x| [x.name,x.runner,x.output] }

data-import-block 15263213 - 15270000
panworker-main2-28
2022-02-17 13:59:47 +0000 runner panworker-main2-28 take task 2899 : data-import-block 15263213 - 15270000
2022-02-17 13:59:47 +0000 == begin run ==
Exception Class: Net::OpenTimeout
Exception Message: Net::OpenTimeout
Exception Backtrace:
/usr/local/lib/ruby/2.7.0/net/protocol.rb:41:in `ssl_socket_connect'
/usr/local/lib/ruby/2.7.0/net/http.rb:1009:in `connect'
/usr/local/lib/ruby/2.7.0/net/http.rb:943:in `do_start'
/usr/local/lib/ruby/2.7.0/net/http.rb:932:in `start'
/usr/local/lib/ruby/2.7.0/net/http.rb:1483:in `request'
/usr/local/bundle/gems/ethereum.rb-2.5/lib/ethereum/http_client.rb:33:in `send_single'
/usr/local/bundle/gems/ethereum.rb-2.5/lib/ethereum/client.rb:126:in `send_command'
/usr/local/bundle/gems/ethereum.rb-2.5/lib/ethereum/client.rb:137:in `block (2 levels) in <class:Client>'
/panworker/task.rb:193:in `block in data_import_block'
/usr/local/bundle/gems/parallel-1.21.0/lib/parallel.rb:515:in `call_with_index'
/

In [ ]:
puts Task.where(status:"close").map {|x| x.output}

2022-02-16 11:14:24 +0000 runner panworker-main2-13 take task 2892 : hello world task-3
2022-02-16 11:14:24 +0000 == begin run ==
hello world!
2022-02-16 11:14:24 +0000 == end run ==
2022-02-16 11:14:21 +0000 runner panworker-main2-13 take task 2889 : hello world task-0
2022-02-16 11:14:21 +0000 == begin run ==
hello world!
2022-02-16 11:14:21 +0000 == end run ==
2022-02-16 11:14:26 +0000 runner panworker-main2-28 take task 2893 : hello world task-4
2022-02-16 11:14:26 +0000 == begin run ==
hello world!
2022-02-16 11:14:26 +0000 == end run ==
2022-02-16 11:14:34 +0000 runner panworker-main2-13 take task 2896 : hello world task-7
2022-02-16 11:14:34 +0000 == begin run ==
hello world!
2022-02-16 11:14:34 +0000 == end run ==
2022-02-16 11:14:40 +0000 runner panworker-main2-28 take task 2898 : hello world task-9
2022-02-16 11:14:40 +0000 == begin run ==
hello world!
2022-02-16 11:14:40 +0000 == end run ==
2022-02-16 11:14:21 +0000 runner panworker-main2-28 take task 2890 : hello world task

[]

In [ ]:
task_hello_task(10)

10

In [ ]:
def task_ping(count)
  count.times {|x|
    code = <<~TASKCODE

    puts "ping"
    
    TASKCODE
    Task.create_task("ping task-#{x}", code)
  }
end

Task.where("name like ?","ping task%").destroy_all
task_ping(50)

50

In [ ]:
puts Task.all.map {|x| [x.status,x.name,x.runner]}.join("\n")

close
hello world task-0
panworker-main2-28
close
hello world task-6
panworker-main2-13
close
hello world task-9
panworker-main2-28
close
hello world task-8
panworker-main2-13
close
hello world task-1
panworker-main2-28
close
hello world task-3
panworker-main2-13
close
hello world task-5
panworker-main2-28
close
hello world task-7
panworker-main2-28
close
hello world task-2
panworker-main2-28
close
hello world task-4
panworker-main2-28


In [ ]:
Task.where("name like ?","ping task%").where(status:"close").where("created_at >= ?",Time.now()-300).select(:runner).distinct(:runner).map {|x| x.runner}

["panworker-main2-13", "panworker-main2-28"]

In [ ]:
code = <<~TASKCODE
    require 'method_source'

    module LocalRunner
        def _log(str)
            print str
        end
    end

    class JobBase
        include LocalRunner
        def self.main_source
            self.instance_method(:main).source
        end
    end

    class MyHelloJob < JobBase
        def main()
            _log ("hello world\n")
            return "value"
        end
    end
TASKCODE

filename = "job.rb"
file = open(filename,"w")
file.write(code)
file.close
load (filename)


job = MyHelloJob.new()
job.main()

Task.create_task("MyHelloJob", MyHelloJob.main_source)


hello world


true

In [ ]:
Task.all

#<ActiveRecord::Relation [#<Task id: 7478, name: "MyHelloJob", status: "close", code: "    def main()\n        _log (\"hello world\n\")\n     ...", output: "2022-02-20 00:35:30 +0000 runner main take task 74...", runner: "main", created_at: "2022-02-20 00:29:29.207230000 +0000", updated_at: "2022-02-20 00:35:30.651661000 +0000", run_timestamp: "2022-02-20 00:35:30.607517000 +0000", return: "value">]>

In [ ]:
Task.destroy_all

[#<Task id: 7476, name: "MyHelloJob", status: "open", code: "    def main()\n        self._log (\"hello world\n\")\n...", output: nil, runner: nil, created_at: "2022-02-20 00:28:51.419154000 +0000", updated_at: "2022-02-20 00:28:51.419154000 +0000", run_timestamp: nil, return: nil>, #<Task id: 7477, name: "MyHelloJob", status: "open", code: "    def main()\n        self._log (\"hello world\n\")\n...", output: nil, runner: nil, created_at: "2022-02-20 00:29:15.310541000 +0000", updated_at: "2022-02-20 00:29:15.310541000 +0000", run_timestamp: nil, return: nil>]

In [ ]:
eval(MyHelloJob.main_source + "main()")

NoMethodError: undefined method `_log' for #<Object:0x000055856cf1d258>

In [ ]:
MyHelloJob.main_source + "main()"

"    def main()\n        _log (\"hello world\n\")\n        return \"value\"\n    end\nmain()"

In [ ]:
def a
 return 3
end

:a

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ebde59c2-d67b-4e31-ad3b-1bdd07014691' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>